In [1]:
from Templates import *
from Utils import *
import h5py
import torch 
import numpy as np
from PIL import Image
import io
import matplotlib.pyplot as plt
import random
from torch.utils.data import Dataset, DataLoader


from torch.autograd import Variable
from __future__ import print_function
%matplotlib inline
import argparse
import os
import random
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import sys, os
from easydict import EasyDict as edict

In [6]:
path = './'
# dSet = CustomDataset(file = '/content/drive/MyDrive/Colab Notebooks/Project/flowers.hdf5',splitType='train')
# trainloader = DataLoader(dSet,batch_size = batch_size,shuffle=True,num_workers = workers)
dSet_test = CustomDataset(file = path + 'dataset/flowers.hdf5',splitType= 'test')
testloader = DataLoader(dSet_test,batch_size = 20 ,shuffle = True,num_workers =4)

In [7]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

params = edict()
params.Ng = 128
params.Np = 1024 
params.Nz = 100
params.Mg = 16
params.Md = 4
params.Nd = 128
params.Wo = 64
params.Ho = 64
params.W = 256
params.H = 256
params.epochs = 30
params.batch_size = 64
params.G_lr = 0.0002
params.D_lr = 0.0002
params.KL = 2.0
params.stage = 1
params.res_n = 4
params.trained = False
params.device = device


In [8]:
netG,netD = load_models(path+'pretrained/',params)

optimizerD = optim.Adam(netD.parameters(), lr=params.D_lr, betas=(0.5, 0.999))
netG_para = []
for p in netG.parameters():
    if p.requires_grad:
        netG_para.append(p)
optimizerG = optim.Adam(netG_para,lr=params.G_lr,  betas=(0.5, 0.999))

In [9]:
if params.stage == 1:
    rImg = 'rImg'; wImg = 'wImg'; gSave = 'G1' ; dSave = 'D1'
elif params.stage == 2:
    rImg = 'rImgB'; wImg = 'wImgB'; gSave = 'G1' ; dSave = 'D1'
G_trainingError = []

for e in range(params.epochs):
    for i, data in enumerate(testloader, 0):
        # Training
        right_images = Variable(data[rImg].float()).to(device)
        wrong_images = Variable(data[wImg].float()).to(device)
        right_embeds  = Variable(data['rEmbed'].float()).to(device)
        real_labels  = torch.full((right_images.size(0),), 1, dtype=torch.float, device=device)
        fake_labels = torch.full((right_images.size(0),), 0,dtype=torch.float,device= device)
        noise = torch.randn((right_images.size(0),params.Nz),device = device)    

        netD.zero_grad()
        _, gen_imgs, mu, logvar = netG(right_embeds,noise)
        
        errD, errD_real, errD_wrong, errD_fake = netD_loss(netD, right_images, gen_imgs, real_labels, fake_labels, right_embeds)
        errD.backward()
        optimizerD.step()

        netG.zero_grad()

        errG = netG_loss(netD, gen_imgs, real_labels, right_embeds)
        kl_loss = KL_Loss(mu,logvar)
        errG_total = errG + kl_loss + params.KL
        errG_total.backward()
        optimizerG.step()
        if i%10 == 0:
            log_result(e, i, errD, errG_total)
        if i%100:
            modelG_dict = {'state_dict':netG.state_dict(),'epoch': e,'idx': i} 
            save_model(path+ 'pretrained/', modelG_dict, fName=gSave)
            modelD_dict = {'state_dict':netD.state_dict()} 
            save_model(path+ 'pretrained/', modelD_dict, fName=dSave)

Epoch 0, Iter: 0, netDLoss: 1.090720, netGLoss: 3.761497
G1 saved @ ./pretrained/G1.pth
D1 saved @ ./pretrained/D1.pth
G1 saved @ ./pretrained/G1.pth
D1 saved @ ./pretrained/D1.pth
G1 saved @ ./pretrained/G1.pth
D1 saved @ ./pretrained/D1.pth
G1 saved @ ./pretrained/G1.pth
D1 saved @ ./pretrained/D1.pth
G1 saved @ ./pretrained/G1.pth
D1 saved @ ./pretrained/D1.pth
G1 saved @ ./pretrained/G1.pth
D1 saved @ ./pretrained/D1.pth
G1 saved @ ./pretrained/G1.pth
D1 saved @ ./pretrained/D1.pth
G1 saved @ ./pretrained/G1.pth
D1 saved @ ./pretrained/D1.pth
G1 saved @ ./pretrained/G1.pth
D1 saved @ ./pretrained/D1.pth
Epoch 0, Iter: 10, netDLoss: 0.819667, netGLoss: 4.074811
G1 saved @ ./pretrained/G1.pth
D1 saved @ ./pretrained/D1.pth


KeyboardInterrupt: 

In [6]:
i,data = next(enumerate(testloader))
# Training
plt.rcParams["figure.figsize"] = (20,5)


(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)


In [15]:
gen_imgs

torch.Size([6, 3, 64, 64])

In [8]:
i,data = next(enumerate(testloader))
data['rImgB'].shape

(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)


torch.Size([6, 3, 256, 256])